<a href="https://colab.research.google.com/github/carlogalli/colab-gpu/blob/main/arellano-longtermdebt-tasteshocks-gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

Start by running the cell below.
You will get warning "", don't worry and continue to the following cell.

In [ ]:
# Installation cell
%%capture
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget -q 'https://julialang-s3.julialang.org/bin/linux/x64/1.7/julia-1.7.2-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'
echo 'Done'

After you have run the Installation cell (the cell directly above this text), go to Colab's menu bar and select **Edit** and select **Notebook settings** from the drop down. Select *Julia 1.7* in Runtime type. You can also select your prefered harwdware acceleration (defaults to GPU).

You will first get warning "Runtime disconnected", and then notification "Connected to julia-1.7 Google Compute Engine backend (GPU)".

In [ ]:
# This is Julia code: it prints the current version of Julia
# Run it to check you are effectively using Julia now
VERSION

v"1.7.2"

In [ ]:
# Add the CUDA package. This will take some time, around 2 minutes
using Pkg
Pkg.add(["CUDA", "Random", "Distributions", "Printf", "PyPlot", "PrettyTables", "Adapt", "DataFrames", "CSV", "Interpolations"]);
ENV["JULIA_CUDA_USE_BINARYBUILDER"] = false
using Random, Distributions, CUDA, Printf, PyPlot, PrettyTables, Adapt, DataFrames, CSV, Interpolations;

# Print the properties of the GPU in use
function print_gpu_properties()

    for (i,device) in enumerate(CUDA.devices())
        println("*** General properties for device $i ***")
        name = CUDA.name(device)
        println("Device name: $name")
        major = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_COMPUTE_CAPABILITY_MAJOR)
        minor = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_COMPUTE_CAPABILITY_MINOR)
        println("Compute capabilities: $major.$minor")
        clock_rate = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_CLOCK_RATE)
        println("Clock rate: $clock_rate")
        device_overlap = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_GPU_OVERLAP)
        print("Device copy overlap: ")
        println(device_overlap > 0 ? "enabled" : "disabled")
        kernel_exec_timeout = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_KERNEL_EXEC_TIMEOUT)
        print("Kernel execution timeout: ")
        println(kernel_exec_timeout > 0 ? "enabled" : "disabled")
        # a = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_MAX_BLOCK_DIM_X)
        # d = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_MAX_GRID_DIM_X)
        a = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_MULTIPROCESSOR_COUNT)
        println("Number of multiprocessors: $a")
        b = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_MAX_BLOCKS_PER_MULTIPROCESSOR)
        println("Max blocks per MP: $b")
        c = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_MAX_THREADS_PER_BLOCK)
        println("Max threads per block: $c")

        println([a b c a*b*c])
    end
end
print_gpu_properties()

# Structures and Functions

Structures
- ```ModelParams```: parameters of the model
- ```ModelArrays```: value and policy functions
- ```ModelArraysTemp```: arrays we fill when computing choice probabilities
- ```PricesEqm```: equilibrium debt prices as a function of the state variables
- ```PricesFuture```: debt price schedules as a function of future debt choices
- ```SimulationArrays```: arrays where we will store the simulation allocations
- ```SimulationPrices```: arrays where we will store the simulation prices


In [ ]:
# Forbids use to use scalar indexing in GPU arrays
CUDA.allowscalar(false)

# Model Parameters
struct ModelParams
    # Array Parameters
    nb::Int32
    ny::Int32
    T_longterm::Int32
    T_sim::Int32
    T_discard_sim::Int32
    bgrid_lb::Float32
    bgrid_ub::Float32

    # Model Parameters
    σ_c::Float32    # c risk aversion
    r::Float32      # intnl risk-free rate
    ρ::Float32      # income process persistence
    σ_ϵ::Float32    # income process error standard deviation
    θ::Float32      # ree-enter probability
    h::Float32      # haircut on debt

    β::Float32      # df govt
    d0::Float32     # linear coefficient on default costadd C
    d1::Float32     # quadratic coefficient on default cost
    λ::Float32      # debt maturity
    κ::Float32      # debt coupon
    ρ_B::Float32    # coefficient of Gumbel debt taste shocks
    ρ_δ::Float32    # coefficient of Gumbel default taste shocks

    gpu::Bool
end

function build_ModelParams_fn(;
        nb=51, ny=21, T_longterm=20, T_sim=10^6, T_discard_sim=10^3, bgrid_lb=1e-2, bgrid_ub=2.5,
        σ_c=2.0, r=0.00598, ρ=0.9293, σ_ϵ=0.0115, θ=0.282, h=0.37,
        β=0.95, dtilde=0.02, dratio_coeff=0.5, λ=1.0, κ=0.0,
        ρ_B=1e-10, ρ_δ=1e-10,
        gpu=true
    )

    σ_y = σ_ϵ / sqrt(1 - ρ^2)
    μ = 0.0f0
    n_std = 4
    grid_ub = μ + n_std * σ_y
    y_ub = exp(grid_ub)
    dratio_ub = ((y_ub^2) * (1 + dtilde) - y_ub) / (dtilde * (2 * y_ub - 1))
    dratio = dratio_ub * dratio_coeff
    d1 = dtilde * (dratio - y_ub) / (y_ub^2 - y_ub)
    d0 = dtilde * (y_ub^2 - dratio) / (y_ub^2 - y_ub)

    println(σ_y)
    # println(dratio_ub)
    # println([d0+d1, (d0*y_ub+d1*y_ub^2)/y_ub])

    return ModelParams(
        nb, ny, T_longterm, T_sim, T_discard_sim, bgrid_lb, bgrid_ub,
        σ_c, r, ρ, σ_ϵ, θ, h,
        β, d0, d1, λ, κ,
        ρ_B, ρ_δ,
        gpu
    )
end
function convert_ModelParams_cpu(mp::ModelParams)
    return build_ModelParams(
        nb=mp.nb, ny=mp.ny, T_longterm=mp.T_longterm, T_sim=mp.T_sim, T_discard_sim=mp.T_discard_sim, bgrid_lb=mp.bgrid_lb, bgrid_ub=mp.bgrid_ub,
        σ_c=mp.σ_c, r=mp.r, ρ=mp.ρ, σ_ϵ=mp.σ_ϵ, θ=mp.θ, h=mp.h,
        β=mp.β,
        d0=mp.d0, d1=mp.d1, λ=mp.λ, κ=mp.κ,
        ρ_B=mp.ρ_B, ρ_δ=mp.ρ_δ,
        gpu=false
    )
end

# Model Arrays
struct ModelArrays{V,A}
    bgrid::V
    ygrid::V
    P::A

    v0::A
    v::A
    ev::A
    evd::A
    def_policy::A

    vr::A
    c_rep::A
    bprime_rep::A
    q_rep::A
    qfuture_rep::A

    vd::A
    q_def::A
    qfuture_def::A
end
function build_ModelArrays(mp::ModelParams)
    # Bond grid
    bgrid = collect(range(mp.bgrid_lb, stop=mp.bgrid_ub, length=mp.nb))

    # Endowment grid and transition probs
    grid, P = tauchen_carlo(mp.ny, mp.ρ, mp.σ_ϵ)
    ygrid = exp.(grid)

    yh = ygrid[end]
    if 1 - mp.d0 - mp.d1 * 2 * yh < 0
        error("default costs make yd decreasing in y")
    end

    if mp.gpu
        bgrid = CuArray(bgrid)
        ygrid = CuArray(ygrid)
        P = CuArray(P)
        v0, v, ev, evd, def_policy = [CUDA.zeros(mp.nb, mp.ny) for i in 1:5]
        vr, c_rep, bprime_rep, q_rep, qfuture_rep = [CUDA.zeros(mp.nb, mp.ny) for i in 1:5]
        vd, q_def, qfuture_def = [CUDA.zeros(mp.nb, mp.ny) for i in 1:3]
    else
        v0, v, ev, evd, def_policy = [zeros(Float32, mp.nb, mp.ny) for i in 1:5]
        vr, c_rep, bprime_rep, q_rep, qfuture_rep = [zeros(Float32, mp.nb, mp.ny) for i in 1:5]
        vd, q_def, qfuture_def = [zeros(Float32, mp.nb, mp.ny) for i in 1:3]
    end

    return ModelArrays(
        bgrid, ygrid, P,
        v0, v, ev, evd, def_policy,
        vr, c_rep, bprime_rep, q_rep, qfuture_rep,
        vd, q_def, qfuture_def
    )
end
function Adapt.adapt_structure(to, model::ModelArrays)
    bgrid = adapt(to, model.bgrid)
    ygrid = adapt(to, model.ygrid)
    P = adapt(to, model.P)

    v0 = adapt(to, model.v0)
    v = adapt(to, model.v)
    ev = adapt(to, model.ev)
    evd = adapt(to, model.evd)
    def_policy = adapt(to, model.def_policy)

    vr = adapt(to, model.vr)
    c_rep = adapt(to, model.c_rep)
    bprime_rep = adapt(to, model.bprime_rep)
    q_rep = adapt(to, model.q_rep)
    qfuture_rep = adapt(to, model.qfuture_rep)

    vd = adapt(to, model.vd)
    q_def = adapt(to, model.q_def)
    qfuture_def = adapt(to, model.qfuture_def)

    ModelArrays(
        bgrid, ygrid, P,
        v0, v, ev, evd, def_policy,
        vr, c_rep, bprime_rep, q_rep, qfuture_rep,
        vd, q_def, qfuture_def
    )
end

# Model Arrays (temporary)
struct ModelArraysTemp{A}
    vs_rep::A
    cs_rep::A
    cps_rep::A
end
function build_ModelArraysTemp(mp::ModelParams)
    if mp.gpu
        vs_rep, cs_rep, cps_rep = [CUDA.zeros(mp.nb, mp.ny, mp.nb) for i in 1:3]
    else
        vs_rep, cs_rep, cps_rep = [zeros(Float32, mp.nb, mp.ny, mp.nb) for i in 1:3]
    end

    return ModelArraysTemp(
        vs_rep, cs_rep, cps_rep
    )
end
function Adapt.adapt_structure(to, model::ModelArraysTemp)
    vs_rep = Adapt.adapt_structure(to, model.vs_rep)
    cs_rep = Adapt.adapt_structure(to, model.cs_rep)
    cps_rep = Adapt.adapt_structure(to, model.cps_rep)

    ModelArraysTemp(
        vs_rep, cs_rep, cps_rep
    )
end

# Equilibrium Prices
struct PricesEqm{A}
    expdef_short_eqm_rep::A
    expdef_long_eqm_rep::A
end
function build_PricesEqm(mp::ModelParams)
    if mp.gpu
        expdef_short_eqm_rep, expdef_long_eqm_rep = [CUDA.zeros(Float32, mp.nb, mp.ny) for i in 1:2]
    else
        expdef_short_eqm_rep, expdef_long_eqm_rep = [zeros(Float32, mp.nb, mp.ny) for i in 1:2]
    end

    return PricesEqm(
        expdef_short_eqm_rep, expdef_long_eqm_rep
    )
end
function Adapt.adapt_structure(to, model::PricesEqm)
    expdef_short_eqm_rep = Adapt.adapt_structure(to, model.expdef_short_eqm_rep)
    expdef_long_eqm_rep = Adapt.adapt_structure(to, model.expdef_long_eqm_rep)

    PricesEqm(
        expdef_short_eqm_rep, expdef_long_eqm_rep
    )
end

# Future Prices
struct PricesFuture{A}
    expdef_short_future_rep::A
    expdef_long_future_rep::A
end
function build_PricesFuture(mp::ModelParams)
    if mp.gpu
        expdef_short_future_rep, expdef_long_future_rep = [CUDA.zeros(Float32, mp.nb, mp.ny) for i in 1:2]
    else
        expdef_short_future_rep, expdef_long_future_rep = [zeros(Float32, mp.nb, mp.ny) for i in 1:2]
    end

    return PricesFuture(
        expdef_short_future_rep, expdef_long_future_rep
    )
end
function Adapt.adapt_structure(to, model::PricesFuture)
    expdef_short_future_rep = Adapt.adapt_structure(to, model.expdef_short_future_rep)
    expdef_long_future_rep = Adapt.adapt_structure(to, model.expdef_long_future_rep)

    PricesFuture(
        expdef_short_future_rep, expdef_long_future_rep
    )
end

# Simulation Arrays
struct SimulationArrays{Vf,Vi}
    y_sim::Vf
    b_sim::Vf
    q_sim::Vf
    c_sim::Vf

    def_status::Vi
    def_new::Vi

    b_sim_indices::Vi
    y_sim_indices::Vi

    taste_errors::Vf
end
function build_SimulationArrays(T_sim::Int32)
    y_sim, b_sim, q_sim, c_sim = [zeros(Float32, T_sim) for i in 1:4]
    def_status, def_new = [zeros(Int32, T_sim) for i in 1:2]
    b_sim_indices, y_sim_indices = [zeros(Int32, T_sim) for i in 1:2]
    taste_errors = zeros(Float32, 2)

    return SimulationArrays(
        y_sim, b_sim, q_sim, c_sim,
        def_status, def_new,
        b_sim_indices, y_sim_indices,
        taste_errors
    )
end
function Adapt.adapt_structure(to, model::SimulationArrays)
    y_sim = Adapt.adapt_structure(to, model.y_sim)
    b_sim = Adapt.adapt_structure(to, model.b_sim)
    q_sim = Adapt.adapt_structure(to, model.q_sim)
    c_sim = Adapt.adapt_structure(to, model.c_sim)

    def_status = Adapt.adapt_structure(to, model.def_status)
    def_new = Adapt.adapt_structure(to, model.def_new)

    b_sim_indices = Adapt.adapt_structure(to, model.b_sim_indices)
    y_sim_indices = Adapt.adapt_structure(to, model.y_sim_indices)
    taste_errors = Adapt.adapt_structure(to, model.taste_errors)

    SimulationArrays(
        y_sim, b_sim, q_sim, c_sim,
        def_status, def_new,
        b_sim_indices, y_sim_indices,
        taste_errors
    )
end

# Simulation Prices
struct SimulationPrices{V}
    expdef_short_sim::V
    expdef_long_sim::V
end
function build_SimulationPrices(T_sim::Int32; gpu::Bool=false)
    if gpu
        expdef_short_sim, expdef_long_sim = [CUDA.zeros(Float32, T_sim) for i in 1:2]
    else
        expdef_short_sim, expdef_long_sim = [zeros(Float32, T_sim) for i in 1:2]
    end

    return SimulationPrices(
        expdef_short_sim, expdef_long_sim
    )
end
function Adapt.adapt_structure(to, model::SimulationPrices)
    expdef_short_sim = Adapt.adapt_structure(to, model.expdef_short_sim)
    expdef_long_sim = Adapt.adapt_structure(to, model.expdef_long_sim)

    SimulationPrices(
        expdef_short_sim, expdef_long_sim
    )
end

Functions to solve the model

In [ ]:
# Returns AR(1) transition matrix and grid for output
function tauchen_carlo(N::Int32, ρ::Float32, σ_ϵ::Float32; μ::Real=0, n_std::Real=4)

    # Process (x_t above) standard deviation (std is ϵ's standard deviation)
    σ_y = σ_ϵ / sqrt(1 - ρ^2)
    grid_bar = n_std * σ_y      # grid bounds
    grid = collect(Float32, range(μ - grid_bar, stop=μ + grid_bar, length=N))
    dif = grid[2] - grid[1]

    # Get transition probabilities
    P = zeros(Float32, N, N)

    # return nothing

    for row in 1:N

        # do endpoints first
        P[row, 1] = cdf(Normal(0.0, 1.0), (grid[1] + dif / 2 - (1 - ρ) * μ - ρ * grid[row]) / σ_ϵ)
        P[row, N] = 1 - cdf(Normal(0.0, 1.0), (grid[N] - dif / 2 - (1 - ρ) * μ - ρ * grid[row]) / σ_ϵ)

        # middle columns
        for col in 2:N-1
            P[row, col] = cdf(Normal(0.0, 1.0), (grid[col] + dif / 2 - (1 - ρ) * μ - ρ * grid[row]) / σ_ϵ) - cdf(Normal(0.0, 1.0), (grid[col] - dif / 2 - (1 - ρ) * μ - ρ * grid[row]) / σ_ϵ)
        end
    end

    # normalize
    sums = sum(P, dims=2)

    P = @. P / sums

    if maximum(abs.(sum(P, dims=2) .- 1)) > 1e-6
        error("Matrix rows must sum up to 1!")
    end

    return grid, P
end

# Computes utility from consumption
function u_typed_fn(mp::ModelParams, c::Float32)
    uc = (c^(1.0f0 - mp.σ_c)) / (1.0f0 - mp.σ_c)
    return uc
end

# Computes output when in default
function ydef_fn(mp::ModelParams, y::Float32)
    return y - max(0, mp.d0 * y + mp.d1 * y^2)
end

# Creates initial guess for the value and policy functions
function model_init!(mp::ModelParams, ma::ModelArrays)

    if mp.gpu
        b0 = (blockIdx().x - 1) * blockDim().x + threadIdx().x
        y0 = (blockIdx().y - 1) * blockDim().y + threadIdx().y
        if b0 <= mp.nb && y0 <= mp.ny
            model_init!(mp, ma, b0, y0)
        end
    else
        for b0 in 1:mp.nb, y0 in 1:mp.ny
            model_init!(mp, ma, b0, y0)
        end
    end

    return nothing
end
function model_init!(mp::ModelParams, ma::ModelArrays, b0::Int, y0::Int)
    y = ma.ygrid[y0]
    yd = ydef_fn(mp, y)
    b = ma.bgrid[b0]

    #=
    Repayment: in the last period, max U(c) s.t. c = y-b
    =#
    ma.c_rep[b0, y0] = y - b
    ma.vr[b0, y0] = u_typed_fn(mp, ma.c_rep[b0, y0])
    ma.bprime_rep[b0, y0] = 0.0
    ma.q_rep[b0, y0] = 1.0

    #=
    Default: in the last period, max U(c) s.t. c = yd-b(1-h)
    =#
    cd = yd - b * (1 - mp.h)
    ma.vd[b0, y0] = u_typed_fn(mp, cd)
    ma.q_def[b0, y0] = 1.0

    return nothing
end

# Uses repay and default value fns to update default policy and main value fn
function update_values_expectations_1!(mp::ModelParams, ma::ModelArrays)
    # uses: vr, vd
    # updates: def_policy, v

    if mp.gpu
        b0 = (blockIdx().x - 1) * blockDim().x + threadIdx().x
        y0 = (blockIdx().y - 1) * blockDim().y + threadIdx().y
        if b0 <= mp.nb && y0 <= mp.ny
            uve_1!(mp, ma, b0, y0)
        end
    else
        for b0 in 1:mp.nb, y0 in 1:mp.ny
            uve_1!(mp, ma, b0, y0)
        end
    end

    return nothing
end
function uve_1!(mp::ModelParams, ma::ModelArrays, b0::Int, y0::Int)
    if exp((ma.vd[b0, y0] - ma.vr[b0, y0]) / mp.ρ_δ) == Inf
        ma.def_policy[b0, y0] = 1.0
    else
        ma.def_policy[b0, y0] = exp((ma.vd[b0, y0] - ma.vr[b0, y0]) / mp.ρ_δ) / (1 + exp((ma.vd[b0, y0] - ma.vr[b0, y0]) / mp.ρ_δ))
    end
    ma.v[b0, y0] = ma.def_policy[b0, y0] * ma.vd[b0, y0] + (1 - ma.def_policy[b0, y0]) * ma.vr[b0, y0]

    return nothing
end

# Uses eqm prices and value functions to update expected value fns and future prices
function update_values_expectations_2!(mp::ModelParams, ma::ModelArrays)
    # uses: bgrid, v, vd, P, def_policy, c_rep, Π_rep, q_rep, c_def, Π_def, q_def
    # updates: ev, evd, qfuture_rep, qfuture_def

    if mp.gpu
        b0 = (blockIdx().x - 1) * blockDim().x + threadIdx().x
        y0 = (blockIdx().y - 1) * blockDim().y + threadIdx().y
        if b0 <= mp.nb && y0 <= mp.ny
            uve_2!(mp, ma, b0, y0)
        end
    else
        for b0 in 1:mp.nb, y0 in 1:mp.ny
            uve_2!(mp, ma, b0, y0)
        end
    end

    return nothing
end
function uve_2!(mp::ModelParams, ma::ModelArrays, b0::Int, y0::Int)
    b1 = b0
    b1_def = b1     # to keep things clearer when computing qfuture_def
    b1_def_hc = CUDA.min(CUDA.searchsortedfirst(ma.bgrid, ma.bgrid[b1_def] * (1 - mp.h)), mp.nb)

    # Reset values to zero
    ma.ev[b1, y0], ma.evd[b1, y0] = 0.0, 0.0
    ma.qfuture_rep[b1, y0], ma.qfuture_def[b1, y0] = 0.0, 0.0
    for y1 in 1:mp.ny

        # compute expected value functions, makes them functions of (̃B',y)
        ma.ev[b1, y0] += ma.v[b1, y1] * ma.P[y0, y1]
        ma.evd[b1, y0] += ma.vd[b1, y1] * ma.P[y0, y1]

        #= the update of the qfuture price fns should only use q_rep and q_def, not qfuture itself! =#

        δprime = ma.def_policy[b1, y1]
        δprime_hc = ma.def_policy[b1_def_hc, y1]

        # uses: def_policy, q_rep, Π_rep, q_def, Π_def
        ma.qfuture_rep[b1, y0] += (
            (1 - δprime) * (mp.λ + (1 - mp.λ) * (mp.κ + ma.q_rep[b1, y1])) +
            +δprime * ma.q_def[b1, y1]
        ) * ma.P[y0, y1] / (1 + mp.r)

        # uses: q_def, Π_def here, q_rep, Π_rep below
        ma.qfuture_def[b1_def, y0] += (
            (1 - mp.θ) * ma.q_def[b1_def, y1] +
            +mp.θ * (1 - mp.h) * (
                (1 - δprime_hc) * (mp.λ + (1 - mp.λ) * (mp.κ + ma.q_rep[b1_def_hc, y1])) +
                δprime_hc * ma.q_def[b1_def_hc, y1]
            )
        ) * ma.P[y0, y1] / (1 + mp.r)

        # ma.qfuture_def[b1_def, y0] += (
        #     (1-mp.θ)*ma.q_def[b1_def,y1]/ma.Π_def[b1_def,y1] +
        #     mp.θ*(1-mp.h)*(mp.λ+(1-mp.λ)*(mp.κ+ma.q_rep[b1_def_hc,y1]))/ma.Π_rep[b1_def_hc,y1]
        # ) * ma.P[y0,y1] / (1+mp.r)
    end

    return nothing
end

# Solves the repayment problem
function vr_gridsearch!(mp::ModelParams, ma::ModelArrays, mat::ModelArraysTemp)

    if mp.gpu
        b0 = (blockIdx().x - 1) * blockDim().x + threadIdx().x
        y0 = (blockIdx().y - 1) * blockDim().y + threadIdx().y
        if b0 <= mp.nb && y0 <= mp.ny
            vr_gridsearch!(mp, ma, mat, b0, y0)
        end
    else
        for b0 in 1:mp.nb, y0 in 1:mp.ny
            vr_gridsearch!(mp, ma, mat, b0, y0)
        end
    end

    return nothing
end
function vr_gridsearch!(mp::ModelParams, ma::ModelArrays, mat::ModelArraysTemp, b0::Int, y0::Int)
    # Uses:
    # - ma.ev, ma.qfuture_rep

    # Updates:
    # - mat.vs_rep, mat.cps_rep
    # - ma.vr, ma.c_rep, ma.bprime_rep, ma.q_rep

    y = ma.ygrid[y0]
    b = ma.bgrid[b0]
    vstar = -Inf32

    for b1 in 1:mp.nb
        bprime = ma.bgrid[b1]
        qprime = ma.qfuture_rep[b1, y0]

        c = y + qprime * bprime - b * (mp.λ + (1 - mp.λ) * (mp.κ + qprime))

        if c <= 0.0f0
            v = -1.0f3
        else
            v = u_typed_fn(mp, c) + mp.β * ma.ev[b1, y0]
        end

        # v = u_typed_fn(mp, c, g, Πstar, ma.ygrid[y0]) + mp.β * ma.ev[b1, y0]
        vstar = ifelse(v > vstar, v, vstar)

        mat.vs_rep[b0, y0, b1] = v
        mat.cs_rep[b0, y0, b1] = c
    end

    D = 0.0
    for b1 in 1:mp.nb
        D += exp((mat.vs_rep[b0, y0, b1] - vstar) / mp.ρ_B)
    end
    for b1 in 1:mp.nb
        mat.cps_rep[b0, y0, b1] = exp((mat.vs_rep[b0, y0, b1] - vstar) / mp.ρ_B) / D
    end

    ma.vr[b0, y0] = vstar + mp.ρ_B * log(D)

    # Update policy functions
    ma.c_rep[b0, y0], ma.bprime_rep[b0, y0], ma.q_rep[b0, y0] = 0.0, 0.0, 0.0
    for b1 in 1:mp.nb
        cp = mat.cps_rep[b0, y0, b1]

        c = mat.cs_rep[b0, y0, b1]

        ma.c_rep[b0, y0] += cp * c
        ma.bprime_rep[b0, y0] += cp * ma.bgrid[b1]
        ma.q_rep[b0, y0] += cp * ma.qfuture_rep[b1, y0]
    end

    return nothing
end

# Solves the default problem
function vd_gridsearch!(mp::ModelParams, ma::ModelArrays, mat::ModelArraysTemp)
    if mp.gpu
        b0 = (blockIdx().x - 1) * blockDim().x + threadIdx().x
        y0 = (blockIdx().y - 1) * blockDim().y + threadIdx().y
        if b0 <= mp.nb && y0 <= mp.ny
            vd_gridsearch!(mp, ma, mat, b0, y0)
        end
    else
        for b0 in 1:mp.nb, y0 in 1:mp.ny
            vd_gridsearch!(mp, ma, mat, b0, y0)
        end
    end

    return nothing
end
function vd_gridsearch!(mp::ModelParams, ma::ModelArrays, mat::ModelArraysTemp, b0::Int, y0::Int)
    # Uses:
    # - ma.ev, ma.evd, ma.qfuture_def

    # Updates:
    # - mat.vs_def, mat.cps_def
    # - ma.vd, ma.c_def, ma.g_def, ma.Π_def, ma.bprime_def, ma.q_def

    vstar = -Inf32
    yd = ydef_fn(mp, ma.ygrid[y0])
    c = yd

    b1 = b0     # debt tomorrow if no re-entry option tomorrow
    b1_hc = searchsortedfirst(ma.bgrid, ma.bgrid[b0] * (1 - mp.h))      # debt tomorrow if re-entry option tomorrow

    vd = u_typed_fn(mp, c) + mp.β * (
        (1 - mp.θ) * ma.evd[b1, y0] + mp.θ * ma.ev[b1_hc, y0]
    )

    ma.vd[b0, y0] = vd
    ma.q_def[b0, y0] = ma.qfuture_def[b1, y0]

    return nothing
end

# Computes future prices (q(b',y)) using eqm prices (q(b,y))
function compute_prices_future!(mp::ModelParams, ma::ModelArrays, pe::PricesEqm, pf::PricesFuture)
    if mp.gpu
        b1 = (blockIdx().x - 1) * blockDim().x + threadIdx().x
        y0 = (blockIdx().y - 1) * blockDim().y + threadIdx().y
        if b1 <= mp.nb && y0 <= mp.ny
            compute_prices_future!(mp, ma, pe, pf, b1, y0)
        end
    else
        for b1 in 1:mp.nb, y0 in 1:mp.ny
            compute_prices_future!(mp, ma, pe, pf, b1, y0)
        end
    end

    return nothing
end
function compute_prices_future!(mp::ModelParams, ma::ModelArrays, pe::PricesEqm, pf::PricesFuture, b1::Int, y0::Int)
    # Uses eqm prices to compute future prices (that depends on (b',y))
    # uses:
    # - P, def_policy, Π_rep, Π_def
    # - expdef_long_eqm_rep, expinf_long_eqm_rep, expinf_long_eqm_def
    # updates:
    # - expdef_short_future_rep, expdef_long_future_rep, expinf_short_future_rep, expinf_long_future_rep

    b1_def = b1     # to keep things clearer when computing qfuture_def
    b1_def_hc = CUDA.min(CUDA.searchsortedfirst(ma.bgrid, ma.bgrid[b1_def] * (1 - mp.h)), mp.nb)

    pf.expdef_short_future_rep[b1, y0] = 0.0f0
    pf.expdef_long_future_rep[b1, y0] = 0.0f0

    for y1 in 1:mp.ny
        pf.expdef_short_future_rep[b1, y0] += (
            ma.def_policy[b1, y1]
        ) * ma.P[y0, y1]

        pf.expdef_long_future_rep[b1, y0] += (
            ma.def_policy[b1, y1] + (1 - ma.def_policy[b1, y1]) * pe.expdef_long_eqm_rep[b1, y1]
        ) * ma.P[y0, y1]
    end
    return nothing
end

# Computes eqm prices
function compute_prices_eqm!(mp::ModelParams, ma::ModelArrays, mat::ModelArraysTemp, pe::PricesEqm, pf::PricesFuture)
    if mp.gpu
        b0 = (blockIdx().x - 1) * blockDim().x + threadIdx().x
        y0 = (blockIdx().y - 1) * blockDim().y + threadIdx().y
        if b0 <= mp.nb && y0 <= mp.ny
            compute_prices_eqm!(mp, ma, mat, pe, pf, b0, y0)
        end
    else
        for b0 in 1:mp.nb, y0 in 1:mp.ny
            compute_prices_eqm!(mp, ma, mat, pe, pf, b0, y0)
        end
    end

    return nothing
end
function compute_prices_eqm!(mp::ModelParams, ma::ModelArrays, mat::ModelArraysTemp, pe::PricesEqm, pf::PricesFuture, b0::Int, y0::Int)
    # uses:
    # - def_policy, cps_rep, Πs_rep, cs_rep, cps_def
    # - expdef_short_future_rep, expdef_long_future_rep, expinf_short_future_rep, expinf_long_future_rep, expinf_long_future_def
    # updates:
    # - expdef_short_eqm_rep, expdef_long_eqm_rep, expinf_short_eqm_rep, expinf_long_eqm_rep, expinf_long_eqm_def


    # Perform computations
    c_def = ydef_fn(mp, ma.ygrid[y0])

    pe.expdef_short_eqm_rep[b0, y0], pe.expdef_long_eqm_rep[b0, y0] = 0.0, 0.0
    for b1 in 1:mp.nb
        pe.expdef_short_eqm_rep[b0, y0] += mat.cps_rep[b0, y0, b1] * pf.expdef_short_future_rep[b1, y0]
        pe.expdef_long_eqm_rep[b0, y0] += mat.cps_rep[b0, y0, b1] * pf.expdef_long_future_rep[b1, y0]
    end
    return nothing
end

# Main function, takes parameters and runs value fn iteration until convergence
function main_gpu(mp::ModelParams; max_iter::Int=1000, print_iter::Int=0, plot::Int=0, ret_arrays::Bool=false, sim_print::String="no", gpu::Bool=true)

    # println("Check assumption on tax ceiling in vd !!!")

    # Define GPU threads and blocks
    threadsize = min(16, max(mp.nb, mp.ny))
    ts = (threadsize, threadsize)
    blocksize = cld(mp.nb, threadsize)
    bs = (blocksize, blocksize)
    # println([ts, bs])

    # Create ModelArrays object
    ma = build_ModelArrays(mp)
    mat = build_ModelArraysTemp(mp)

    # Initialise everything
    if mp.gpu
        @cuda threads = ts blocks = bs model_init!(mp, ma)
        @cuda threads = ts blocks = bs update_values_expectations_1!(mp, ma)
        @cuda threads = ts blocks = bs update_values_expectations_2!(mp, ma)
    else
        model_init!(mp, ma)
        update_values_expectations_1!(mp, ma)
        update_values_expectations_2!(mp, ma)
    end

    #= Iterative process =#
    err = 1e5
    tol = 1e-4
    iter = 0

    while (err > tol) & (iter < max_iter)
        iter += 1
        ma.v0 .= ma.v

        if mp.gpu
            @cuda threads = ts blocks = bs vr_gridsearch!(mp, ma, mat)
            @cuda threads = ts blocks = bs vd_gridsearch!(mp, ma, mat)
            @cuda threads = ts blocks = bs update_values_expectations_1!(mp, ma)
            @cuda threads = ts blocks = bs update_values_expectations_2!(mp, ma)
        else
            vr_gridsearch!(mp, ma, mat)
            vd_gridsearch!(mp, ma, mat)
            update_values_expectations_1!(mp, ma)
            update_values_expectations_2!(mp, ma)
        end

        err = maximum(@. abs(ma.v - ma.v0))

        if print_iter > 0
            if rem(iter, print_iter) == 0
                println(@sprintf("iter %.0f, error %.1e", iter, err))
            end
        end
    end
    println(@sprintf("vfi ends with %.0f iters, error %.1e", iter, err))


    #= Compute the eqm objects and asset prices =#
    # which were not needed for VFI but are needed for the simulations
    pe = build_PricesEqm(mp)
    pf = build_PricesFuture(mp)

    for i in 1:mp.T_longterm
        if mp.gpu
            @cuda threads = ts blocks = bs compute_prices_future!(mp, ma, pe, pf)       # use eqm to compute future
            @cuda threads = ts blocks = bs compute_prices_eqm!(mp, ma, mat, pe, pf)     # use future to compute eqm
        else
            compute_prices_future!(mp, ma, pe, pf)       # use eqm to compute future
            compute_prices_eqm!(mp, ma, mat, pe, pf)     # use future to compute eqm
        end
    end

    #= Simulation =#
    # Allocate and Pre-Fill
    sa = build_SimulationArrays(mp.T_sim)
    sp = build_SimulationPrices(mp.T_sim)

    #= Convert from GPU to CPU for simulations and plotting =#
    if mp.gpu
        ma = adapt(Array, ma)
        mat = adapt(Array, mat)
        pe = adapt(Array, pe)
        pf = adapt(Array, pf)
    end

    if sim_print != "no"
        # Simulate the economy
        simulate_cg!(mp, ma, mat, pf, sa, sp)

        #= Convert from CPU to GPU to compute moments =#
        if mp.gpu
            sa = adapt(CuArray, sa)
            sp = adapt(CuArray, sp)
        end

        if sim_print in ["table", "small table"]
            # Print tables
            sim_mom!(mp, sa, sp, print_flag=sim_print)

        elseif sim_print in ["moments", "corrs"]
            # Compute a few moments for calibration purposes
            out = sim_mom!(mp, sa, sp, print_flag=sim_print)
            return out
        end
    end

    if plot > 0
        plot_eqm_b(mp, ma, pe, pf; fignum=plot)
        plot_eqm_y(mp, ma, pe, pf, fignum=plot + 1)
    end

    if ret_arrays
        return mp, ma, mat, pe, pf, sa, sp
    else
        return nothing
    end
end

# Runs vfi without the need to create any global variable/structure
function run_main(;
        β=0.9, α_m=20, κ_seign=1, τbar=0.2, dtilde=0.02, dratio_coeff=0.5, sim_print="table"
    )

    mp = build_ModelParams_fn(
        nb=101, ny=35, bgrid_lb=1e-2, bgrid_ub=0.5,
        σ_c=2.0, σ_g=4.0, g_lb=0.0, η=2.0,
        β=β, α_g=3.4e-3, α_m=α_m, κ_seign=κ_seign, τbar=τbar,
        dtilde=dtilde, dratio_coeff=dratio_coeff, λ=1.0, κ=0.0,
        ρ_B=1e-3, ρ_μ=1e-3, ρ_δ=1e-4,
        T_longterm=4,
        gpu=gpu_flag
    )

    main_gpu(mp, max_iter=1000, print_iter=0, sim_print=sim_print)
    nothing
end

run_main (generic function with 1 method)

Functions that simulate the model and compute moments

In [ ]:
# Generate a sample path of a Markov chain, from a given initial state.
# The resulting vector, of dimension T (number of simulation periods), has the state values of the Markov chain as elements.
function markov_draw!(grid::Vector{Float32}, P::Array{Float32}, out_vector::Vector, T::Int32; ret_indices::Bool=false, init_index::Int=round(Int, length(grid) / 2), RNG::MersenneTwister=MersenneTwister())
    #=
    Uses: P (the discrete Markov transition matrix), grid (the state grid)
    =#
    Q = cumsum(P, dims=2)   # CDF, for each state
    simulated_indices = zeros(Int32, T)
    simulated_indices[1] = init_index
    randdraws = rand(RNG, T)

    # if length(out_vector) !== T
    #     println([length(out_vector), T, length(out_vector) !== T])
    #     error("vector lengths differ")
    # end

    for i in 2:T
        i_prev = simulated_indices[i-1]
        simulated_indices[i] = searchsortedfirst(view(Q, i_prev, :), randdraws[i-1])
    end

    if ret_indices
        out_vector .= simulated_indices
    else
        out_vector .= grid[simulated_indices]
    end

    return nothing
end

# Simulate the model using the sample path for the shocks (output and taste)
function simulate_cg!(mp::ModelParams, ma::ModelArrays, mat::ModelArraysTemp, pf::PricesFuture, sa::SimulationArrays, sp::SimulationPrices;
        y_init=1.0, b_init=0.1,
        T=NaN32, MTnumber=6687
    )

    T = ifelse(isnan(T), mp.T_sim, T)

    #=
    REMEMBER TO DON'T COUNT DEFAULT PERIODS WHEN COMPUTING SPREADS !!!!
    =#

    cps_cum_rep = Array{Float32}(accumulate(+, mat.cps_rep, dims=3))

    #= Get initial indices =#
    y_init_index = min(searchsortedfirst(ma.ygrid, y_init), mp.ny)
    b_init_index = min(searchsortedfirst(ma.bgrid, b_init), mp.nb)

    # Create a MarkovChain
    MT = MersenneTwister(MTnumber)

    # Output shocks
    markov_draw!(ma.ygrid, ma.P, sa.y_sim_indices, T, ret_indices=true, init_index=y_init_index, RNG=MT)

    # Get taste shocks for both [b'] and [μ']
    tasteshocks = rand(MT, T)

    # Get default shocks
    defshocks = rand(MT, T)

    # Get re-entry shocks
    reentryshocks = rand(MT, T)

    # Compute expected value of taste shock "errors", i.e. the average (over iB,iy) expected % utility loss, which is =dot(choiceprobs, log(vs/vstar))
    taste_error_b = mean(sum(mat.cps_rep .* (mat.vs_rep ./ maximum(mat.vs_rep, dims=3) .- 1), dims=3))
    taste_error_δ = mean(@. log(ma.vd / max(ma.vr, ma.vd)) + log(ma.vr / ma.vd) * 1 / (1 + exp((ma.vd - ma.vr) / mp.ρ_δ)))
    sa.taste_errors .= [taste_error_b; taste_error_δ]
    #sa.taste_errors .= [0.; 0.; 0.]

    sa.b_sim_indices[1] = b_init_index
    sa.def_status[1] = false


    # Time simulation
    for t = 1:T-1

        # Get indices of current period
        iyt, ibt = sa.y_sim_indices[t], sa.b_sim_indices[t]
        sa.y_sim[t] = ma.ygrid[iyt]
        sa.b_sim[t] = ma.bgrid[ibt]
        #show t, ibt, iyt
        #show B_sim

        #=
        [1] If you are NOT in default *from the past*
        =#
        if sa.def_status[t] == false

            # check if you want to default today
            default_today = defshocks[t] < ma.def_policy[ibt, iyt]

            # [1a] If you DO want to default today
            if default_today
                sa.def_new[t] = true

                # [1b] If you do NOT default
            else
                ibtnext_rep = searchsortedfirst(view(cps_cum_rep, ibt, iyt, :), tasteshocks[t])

                sa.b_sim_indices[t+1] = ibtnext_rep
                sa.q_sim[t] = ma.qfuture_rep[ibtnext_rep, iyt]
                sa.c_sim[t] = mat.cs_rep[ibt, iyt, ibtnext_rep]

                sp.expdef_short_sim[t] = pf.expdef_short_future_rep[ibtnext_rep, iyt]
                sp.expdef_long_sim[t] = pf.expdef_long_future_rep[ibtnext_rep, iyt]
            end
        end
        # need to close the if and open another one for the change in def_status[t] to be picked up

        #=
        [2] If you are in default
            - either from the past (=> did not get settlement option)
            - or from the present (=> just defaulted)
        =#
        if sa.def_status[t] == true || sa.def_new[t] == true

            sa.y_sim[t] = ydef_fn(mp, ma.ygrid[iyt])     # replaces y_sim[t]
            sa.c_sim[t] = sa.y_sim[t]
            sa.q_sim[t] = ma.qfuture_def[ibt, iyt]

            # actual future b' depends on θ
            if reentryshocks[t] < mp.θ && sa.def_new[t] == false
                # with prob θ get settlement option tomorrow, when not after a fresh default
                sa.def_status[t+1] = false

                # b_sim_indices[t+1] = min(searchsortedfirst(mp.bgrid, mp.bgrid[ibt]*(1-mp.h)/(1+μ_qoq_sim[t])), mp.nb)
                sa.b_sim_indices[t+1] = min(searchsortedfirst(ma.bgrid, sa.b_sim[t] * (1 - mp.h)), mp.nb)
                # debt carried over to face settlement choice tomorrow

            else    # with prob 1-θ no settlement option tomorrow
                sa.def_status[t+1] = true

                # b_sim_indices[t+1] = min(searchsortedfirst(mp.bgrid, mp.bgrid[ibt]/(1+μ_qoq_sim[t])), mp.nb)
                sa.b_sim_indices[t+1] = ibt
            end
        end
    end

    return nothing
end

# Auxiliary function that I use in "sim_mom!" to print moment tables
function formatfn(v, row, col)
    if isa(v, String)
        return v
    else
        if abs(v) < 10^5 && abs(v) >= 1e-2
            return @sprintf("%.4f", v)
        else
            return @sprintf("%.1e", v)
        end
    end
end

# Compute and print simulated moments
function sim_mom!(mp::ModelParams, sa::SimulationArrays, sp::SimulationPrices; fignum::Int=0, print_flag::String="table")

    #= Data Preparation =#
    begin
        taste_error_b, taste_error_δ = [Array(sa.taste_errors)[i] for i in 1:2]
        T_end = mp.T_sim - 100
        T_long = @sprintf("(%.0f yrs)", mp.T_longterm / 4)
        T = T_end - mp.T_discard_sim
        time_counter = 1:1:mp.T_sim

        # Compute repay and default indicators (of length T_sim-T_discard)
        repind = @. sa.def_status == false && sa.def_new == false
        repind = @. repind && mp.T_discard_sim <= time_counter <= T_end
        # indicates with 1 periods with access to markets, i.e. where you enter with good credit standing AND you stay in markets by repaying
        defind = @. repind == false && (mp.T_discard_sim <= time_counter <= T_end)
        T_rep = sum(repind)
        T_def = sum(defind)
        repind_long = @. sa.def_status == false && sa.def_new == false && time_counter <= T_end

        # Compute year-long default and repayment indicators
        repind_yoy = @. repind_long[mp.T_discard_sim:T_end] * repind_long[mp.T_discard_sim-1:T_end-1] * repind_long[mp.T_discard_sim-2:T_end-2] * repind_long[mp.T_discard_sim-3:T_end-3]
        defind_yoy = @. repind_long[mp.T_discard_sim:T_end] + repind_long[mp.T_discard_sim-1:T_end-1] + repind_long[mp.T_discard_sim-2:T_end-2] + repind_long[mp.T_discard_sim-3:T_end-3] == 0
        # if repind_yoy[t]=1, then the govt has NEVER defaulted or been in default between t-4 and t
        # if defind_yoy[t]=1, then the govt has ALWAYS been in default between t-4 and t

        # Show that these 3 sets of events indeed include all possibilities
        #residual_yoy = @. 0 < (repind_long[mp.T_discard_sim:end] + repind_long[mp.T_discard_sim-1:T_end] + repind_long[mp.T_discard_sim-2:end-2] + repind_long[mp.T_discard_sim-3:end-3]) < 4
        #show sum(repind_yoy)+sum(defind_yoy)+sum(residual_yoy), T


        # Create repay and default series
        bprime_sim = [view(sa.b_sim, 2:mp.T_sim); 0.0f0]
        y_rep = sa.y_sim[repind]
        y_def = sa.y_sim[defind]
        b_rep = sa.b_sim[repind]
        b_def = sa.b_sim[defind]
        bprime_rep = bprime_sim[repind]
        bprime_def = bprime_sim[defind]
        c_rep = sa.c_sim[repind]
        c_def = sa.c_sim[defind]
        q_rep = sa.q_sim[repind]
        q_def = sa.q_sim[defind]

        expdef_short_rep = sp.expdef_short_sim[repind]
        expdef_long_rep = sp.expdef_long_sim[repind]

        tb_rep = @. b_rep * (mp.λ + (1 - mp.λ) * (mp.κ + q_rep)) - q_rep * bprime_rep

        # Check how often the govt buys debt back
        net_issuance_rep = @. bprime_rep - b_rep * (1 - mp.λ)
        buyback_freq = sum(net_issuance_rep .< 0) / T_rep

        # Compute moments of macro variables
        sd_y_rep = std(y_rep)
        sd_b_rep = std(b_rep)
        sd_bprime_rep = std(bprime_rep)
        sd_tb_rep = std(tb_rep)

        sd_expdef_short = std(expdef_short_rep)
        sd_expdef_long = std(expdef_long_rep)

        debtgdp_rep = @. b_rep / (y_rep)
        debtgdp_def = @. b_def / (y_def)
        debt_rep = @. b_rep
        debt_def = @. b_def
        consgdp_rep = c_rep ./ y_rep
        consgdp_def = c_def ./ y_def

        newdefault_freq_qtrly = sum(@. sa.def_status == false && sa.def_new == true) / sum(@. sa.def_status == false)
        # newdefault_freq_yrly = 1-(1-newdefault_freq_qtrly)^4
        indefault = sum(@. sa.def_status == true) / T

        # Total spread (default + inflation). Follows Hatchondo and Martinez's (2009) calculations
        ytms = @. (mp.λ + (1 - mp.λ) * mp.κ) / sa.q_sim - mp.λ      # constant yield implied by bond prices
        spread = @. ((1 + ytms) / (1 + mp.r))^4 - 1                 # annualised total spread
        spread_rep = spread[repind]
        mean_spread = mean(spread_rep)
        sd_spread = std(spread_rep)

        # Correlations
        rho_y_cds = cov(y_rep, expdef_long_rep) / (sd_y_rep * sd_expdef_long)
        rho_y_b = cov(y_rep, b_rep) / (sd_y_rep * sd_b_rep)
        rho_y_bprime = cov(y_rep, bprime_rep) / (sd_y_rep * sd_b_rep)
        rho_y_tb = cov(y_rep, tb_rep) / (sd_y_rep * sd_tb_rep)
        rho_y_spread = cov(y_rep, bprime_rep) / (sd_y_rep * sd_bprime_rep)
        # rho_debtgdp_spread = cov(debtgdp_rep, spread_rep)/(sd_debtgdp*sd_spread)
        rho_b_cds = cov(b_rep, expdef_long_rep) / (sd_b_rep * sd_expdef_long)

        # Time spent on bgrid bounds
        #time_lb = sum(b_sim.==mp.bgrid_lb)
        #time_ub = sum(b_sim.==mp.bgrid_ub)
        time_lb_rep = sum(bprime_sim[repind] .== mp.bgrid_lb)
        time_ub_rep = sum(bprime_sim[repind] .== mp.bgrid_ub)
        time_lb_def = sum(bprime_sim[defind] .== mp.bgrid_lb)
        time_ub_def = sum(bprime_sim[defind] .== mp.bgrid_ub)
    end

    #= Data Printing =#
    if print_flag == "moments"
        out = [
            mean(debtgdp_rep) * 100,      # 1: b mean
            std(debtgdp_rep) * 100,
            mean(expdef_long_rep) * 100,  # 3: cds mean
            std(expdef_long_rep) * 100,   # 4: cds sd
            rho_y_cds,
            time_lb_rep / T_rep * 100,
            time_ub_rep / T_rep * 100,
            time_lb_def / T_def * 100,
            time_ub_def / T_def * 100,
            buyback_freq * 100
        ]
        return out

    elseif print_flag == "corrs"
        out = [
            rho_y_cds
            rho_y_bprime
        ]
        return out

    elseif print_flag == "small table"

        column_headers = ["statistic", "value, mean", "stdev"]
        output = [
            "b/y (%)" mean(debtgdp_rep)*100 std(debtgdp_rep)*100
            "___" "" ""
            "default freq %" newdefault_freq_qtrly*100 ""
            "default periods %" indefault*100 ""
            "cds long (%)"*T_long mean(expdef_long_rep)*100 std(expdef_long_rep)*100
            "___" "" ""
            "% bound time rep" time_lb_rep/T_rep*100 time_ub_rep/T_rep*100
            "% bound time def" time_lb_def/T_def*100 time_ub_def/T_def*100
        ]
        pretty_table(output, header=column_headers, formatters=formatfn)

    elseif print_flag == "table"
        # Print out tables
        column_headers = ["statistic", "corr, mean", "stdev"]
        output = [
            "corr(y, cds)" rho_y_cds ""
            "corr(y, b)" rho_y_b ""
            "corr(y, b')" rho_y_bprime ""
            "corr(y, b-qb')" rho_y_tb ""
            "___" "" ""
            "taste error b (% of vr)" taste_error_b*100 ""
            "taste error default (% of v)" taste_error_δ*100 ""
            "___" "" ""
            "b/y rep (%)" mean(debtgdp_rep)*100 std(debtgdp_rep)*100
            "b/y def (%)" mean(debtgdp_def)*100 std(debtgdp_def)*100
            "c/y rep (%)" mean(consgdp_rep)*100 std(consgdp_rep)*100
            "c/y def (%)" mean(consgdp_def)*100 std(consgdp_def)*100
            "___" "" ""
            "% LB time rep" time_lb_rep/T_rep*100 ""
            "% UB time rep" time_ub_rep/T_rep*100 ""
            "% LB time def" time_lb_def/T_def*100 ""
            "% UB time def" time_ub_def/T_def*100 ""
            "% time buyback" buyback_freq*100 ""
            "___" "" ""
            "default freq %" newdefault_freq_qtrly*100 ""
            "default periods %" indefault*100 ""
            "cds short (%) (1 qtr)" mean(expdef_short_rep)*100 std(expdef_short_rep)*100
            "cds long (%)"*T_long mean(expdef_long_rep)*100 std(expdef_long_rep)*100
            "q rep" mean(q_rep) std(q_rep)
            "q def" mean(q_def) std(q_def)
        ]

        #println(
        #    @sprintf("nb=%.0f, ny=%.0f, b∈[%.2f, %.2f]\n", mp.nb, mp.ny, mp.bgrid_lb, mp.bgrid_ub),
        #    @sprintf("σ_c=%.0f, σ_g=%.0f, g_lb=%.2f, η=%.1f, r=%.4f, ρ=%.4f, σ_ϵ=%.4f, θ=%.2f, h=%.2f\n", mp.σ_c, mp.σ_g, mp.g_lb, mp.η, mp.r, mp.ρ, mp.σ_ϵ, mp.θ, mp.h),
        #    @sprintf("β=%.2f, α_m=%.2e, ϕ_m=%.1f, ζ_m=%.0f, κ_seign=%.2f, ζ_seign=%.0f, τbar=%.3f\n", mp.β, mp.α_m, mp.ϕ_m, mp.ζ_m, mp.κ_seign, mp.ζ_seign, mp.τbar),
        #    @sprintf("d0=%.4f, d1=%.4f, λ=%.2f, κ=%.2f\n", mp.d0, mp.d1, mp.λ, mp.κ),
        #    @sprintf("ρ_b=%.2e, ρ_μ=%.2e, ρ_δ=%.2e", mp.ρ_B, mp.ρ_μ, mp.ρ_δ)
        #)

        pretty_table(output, header=column_headers, formatters=formatfn)

        if sum(expdef_short_rep) < 1e-6
            println("Default probs are always zero!")
        end
    end

    return
end

sim_mom! (generic function with 1 method)

# Running the code

Run it and see!

In [ ]:
#= test with CPU first =#

gpu_flag = false

beta = 0.95; dtilde = 0.02; dratio_coeff = 0.3

mp_baseline = build_ModelParams_fn(
    nb=101, ny=35, bgrid_lb=1e-2, bgrid_ub=0.5,
    σ_c=2., β=beta,
    dtilde=dtilde, dratio_coeff=dratio_coeff, λ=1., κ=0.,
    ρ_B=1e-4, ρ_δ=1e-10,
    T_longterm=4,
    gpu=gpu_flag
)

# Run to see performance
# note: the first run takes time due to compilation, run at least twice to see performance
@time main_gpu(mp_baseline, max_iter=1000, print_iter=0, sim_print="no")

0.03113782340843731
vfi ends with 181 iters, error 9.9e-05
vfi ends with 181 iters, error 9.9e-05
  4.748534 seconds (2.29 k allocations: 45.040 MiB)


In [ ]:
#= test with GPU now =#

gpu_flag = true

beta = 0.95; dtilde = 0.02; dratio_coeff = 0.3

mp_baseline = build_ModelParams_fn(
    nb=101, ny=35, bgrid_lb=1e-2, bgrid_ub=0.5,
    σ_c=2., β=beta,
    dtilde=dtilde, dratio_coeff=dratio_coeff, λ=1., κ=0.,
    ρ_B=1e-4, ρ_δ=1e-10,
    T_longterm=4,
    gpu=gpu_flag
)

# Run to see performance
# note: the first run takes time due to compilation, run at least twice to see performance
@time main_gpu(mp_baseline, max_iter=1000, print_iter=0, sim_print="no")

0.03113782340843731
vfi ends with 181 iters, error 9.7e-05
vfi ends with 181 iters, error 9.7e-05
  0.298333 seconds (51.87 k allocations: 45.773 MiB)
